# 🏥 Análisis Geoespacial de Hospitales en Perú
## Notebook 5: Dashboard Streamlit Final

### 🎯 Objetivos:
- Crear dashboard con 3 tabs según requerimientos exactos
- Integrar mapas estáticos de GeoPandas (PNG)
- Embeber mapas interactivos de Folium (HTML)
- Cumplir especificaciones del assignment

### 📱 Estructura requerida:
1. **🗂️ Data Description**: Unit of analysis, sources, filtering rules
2. **🗺️ Static Maps & Department Analysis**: GeoPandas maps + tables
3. **🌍 Dynamic Maps**: Folium choropleth + Lima/Loreto proximity

In [19]:
# === CREA app/app.py CON RUTAS ROBUSTAS Y TABS EXIGIDOS ===
import os
from pathlib import Path

def crear_app_streamlit_final():
    """Crea app.py que cumple exactamente con los requerimientos y usa rutas robustas."""
    
    app_code = r'''
import os
from pathlib import Path
import streamlit as st
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# ----- Rutas robustas -----
APP_DIR  = Path(__file__).resolve().parent      # .../app
ROOT_DIR = APP_DIR.parent                       # .../Hospitals-Access-Peru
DATA_DIR = ROOT_DIR / "data"                    # .../data

def f(rel_path: str) -> str:
    """
    Convierte 'data/archivo.ext' o 'archivo.ext' a ruta absoluta segura.
    Si viene sin prefijo, lo busca en /data.
    """
    p = Path(rel_path)
    if not p.is_absolute():
        p = (ROOT_DIR / rel_path) if rel_path.startswith("data") else (DATA_DIR / rel_path)
    return str(p.resolve())

# Configuración de página
st.set_page_config(
    page_title="Hospitals Access Peru",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

@st.cache_data
def load_data():
    """Carga todos los datos necesarios usando rutas robustas."""
    try:
        hospitales = gpd.read_file(f("data/hospitales_procesados.geojson"))
        try:
            distritos = gpd.read_file(f("data/distritos_con_hospitales.geojson"))
        except Exception:
            distritos = None
        try:
            stats_dept = pd.read_csv(f("data/estadisticas_departamentales.csv"))
        except Exception:
            stats_dept = None
        return hospitales, distritos, stats_dept
    except Exception as e:
        st.error(f"Error cargando datos: {e}")
        return None, None, None

def main():
    """Función principal del dashboard"""
    st.title("🏥 Hospitals Access Peru")
    st.markdown("**Geospatial Analysis of Public Hospital Access**")

    # Cargar datos
    hospitales, distritos, stats_dept = load_data()
    if hospitales is None:
        st.error("❌ No se pudieron cargar los datos. Verifica archivos en la carpeta /data")
        try:
            st.caption("Contenido de /data detectado:")
            st.code("\\n".join(sorted(p.name for p in Path(f('data')).glob('*'))))
        except Exception:
            pass
        return

    # Tabs según requerimientos
    tab1, tab2, tab3 = st.tabs([
        "🗂️ Data Description", 
        "🗺️ Static Maps & Department Analysis", 
        "🌍 Dynamic Maps"
    ])

    with tab1:
        show_data_description(hospitales)

    with tab2:
        show_static_maps_department_analysis(hospitales, stats_dept)

    with tab3:
        show_dynamic_maps()

def show_data_description(hospitales):
    """Tab 1: Data Description con gráficos estadísticos expandidos"""
    
    st.header("📋 Data Description")
    
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.subheader("🎯 Unit of Analysis")
        st.markdown("""
        **Operational public hospitals in Peru**
        
        This analysis focuses exclusively on:
        - Public hospitals that are currently operational
        - Hospitals with valid geographical coordinates (lat/long)
        - Establishments classified as hospitals (not health centers)
        """)
        
        st.subheader("📊 Data Sources")
        
        sources_data = {
            "Dataset": ["MINSA - IPRESS", "Population Centers", "Districts Shapefile"],
            "Source": ["Ministry of Health", "INEI", "Course Repository"],
            "Description": [
                "Operational subset of health establishments",
                "Population centers for proximity analysis", 
                "District boundaries for spatial analysis"
            ],
            "Records": [f"{len(hospitales)} hospitals", "33 centers", "1,873 districts"]
        }
        
        st.dataframe(pd.DataFrame(sources_data), use_container_width=True, hide_index=True)
        
        st.subheader("🔧 Filtering Rules")
        st.markdown("""
        **Only operational hospitals with valid lat/long:**
        
        1. **Institution filter**: Only public institutions (GOBIERNO REGIONAL, MINSA, ESSALUD)
        2. **Status filter**: Only establishments with "EN FUNCIONAMIENTO" condition
        3. **Type filter**: Only "HOSPITALES O CLINICAS" classification
        4. **Geographic filter**: Valid coordinates within Peru boundaries
        5. **Data quality**: Remove records with missing or invalid coordinates
        """)
    
    with col2:
        st.subheader("📈 Key Statistics")
        
        # Métricas principales
        st.metric("Total Hospitals Analyzed", f"{len(hospitales)}")
        st.metric("Departments Covered", f"{hospitales['Departamento'].nunique()}")
        st.metric("Geographic Coverage", "National")
        
        # Distribución por institución
        st.subheader("🏛️ By Institution")
        inst_counts = hospitales['Institución'].value_counts()
        for inst, count in inst_counts.items():
            percentage = (count / len(hospitales)) * 100
            st.write(f"• {inst}: {count} ({percentage:.1f}%)")
        
        # Gráfico de distribución por institución
        fig_pie = px.pie(
            values=inst_counts.values,
            names=inst_counts.index,
            title="Institution Distribution",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_pie.update_traces(textposition='inside', textinfo='percent+label')
        st.plotly_chart(fig_pie, use_container_width=True)

    # SECCIÓN DE GRÁFICOS ESTADÍSTICOS EXPANDIDA
    st.subheader("📊 Statistical Analysis of Hospital Distribution")
    
    # Crear columnas para gráficos
    col1, col2 = st.columns(2)
    
    with col1:
        # Gráfico de barras - Top 10 departamentos
        st.write("**Top 10 Departments by Hospital Count**")
        dept_counts = hospitales['Departamento'].value_counts().head(10)
        
        fig_bar = px.bar(
            x=dept_counts.values,
            y=dept_counts.index,
            orientation='h',
            title="Hospitals per Department",
            labels={'x': 'Number of Hospitals', 'y': 'Department'},
            color=dept_counts.values,
            color_continuous_scale='Blues'
        )
        fig_bar.update_layout(height=400, yaxis={'categoryorder':'total ascending'})
        st.plotly_chart(fig_bar, use_container_width=True)
        
        # Gráfico de clasificación de hospitales
        st.write("**Hospital Classification Distribution**")
        clasif_counts = hospitales['Clasificación'].value_counts()
        
        fig_clasif = px.bar(
            x=clasif_counts.values,
            y=['General' if 'GENERAL' in name else 'Specialized' for name in clasif_counts.index],
            title="Hospital Types",
            labels={'x': 'Count', 'y': 'Type'},
            color=['General', 'Specialized'],
            color_discrete_map={'General': '#1f77b4', 'Specialized': '#ff7f0e'}
        )
        fig_clasif.update_layout(height=300)
        st.plotly_chart(fig_clasif, use_container_width=True)
    
    with col2:
        # Histograma de distribución geográfica
        st.write("**Geographic Distribution - Latitude**")
        fig_hist_lat = px.histogram(
            hospitales, 
            x='latitud' if 'latitud' in hospitales.columns else hospitales.geometry.y,
            nbins=20,
            title="Latitude Distribution",
            labels={'x': 'Latitude', 'y': 'Count'},
            color_discrete_sequence=['#2ca02c']
        )
        fig_hist_lat.update_layout(height=300)
        st.plotly_chart(fig_hist_lat, use_container_width=True)
        
        # Histograma de distribución geográfica - Longitud
        st.write("**Geographic Distribution - Longitude**")
        fig_hist_lon = px.histogram(
            hospitales, 
            x='longitud' if 'longitud' in hospitales.columns else hospitales.geometry.x,
            nbins=20,
            title="Longitude Distribution",
            labels={'x': 'Longitude', 'y': 'Count'},
            color_discrete_sequence=['#d62728']
        )
        fig_hist_lon.update_layout(height=300)
        st.plotly_chart(fig_hist_lon, use_container_width=True)
    
    # Gráfico de dispersión geográfica
    st.write("**Geographic Scatter Plot**")
    
    # Extraer coordenadas según el formato disponible
    if 'latitud' in hospitales.columns and 'longitud' in hospitales.columns:
        lat_col, lon_col = 'latitud', 'longitud'
    else:
        # Si son geometrías de GeoPandas
        hospitales_coords = hospitales.copy()
        hospitales_coords['lat_temp'] = hospitales_coords.geometry.y
        hospitales_coords['lon_temp'] = hospitales_coords.geometry.x
        lat_col, lon_col = 'lat_temp', 'lon_temp'
    
    fig_scatter = px.scatter(
        hospitales,
        x=lon_col,
        y=lat_col,
        color='Departamento',
        title="Hospital Locations Across Peru",
        labels={'x': 'Longitude', 'y': 'Latitude'},
        hover_data=['Nombre del establecimiento', 'Institución'] if 'Nombre del establecimiento' in hospitales.columns else None
    )
    fig_scatter.update_layout(height=500, showlegend=False)  # Hide legend due to many departments
    st.plotly_chart(fig_scatter, use_container_width=True)
    
    # Tabla estadística resumen
    col1, col2 = st.columns(2)
    
    with col1:
        st.write("**Department Statistics Summary**")
        dept_stats = hospitales['Departamento'].value_counts().describe()
        stats_df = pd.DataFrame({
            'Statistic': ['Mean', 'Std Dev', 'Min', '25%', '50%', '75%', 'Max'],
            'Value': [f"{dept_stats['mean']:.1f}", f"{dept_stats['std']:.1f}", 
                     f"{dept_stats['min']:.0f}", f"{dept_stats['25%']:.1f}",
                     f"{dept_stats['50%']:.1f}", f"{dept_stats['75%']:.1f}", 
                     f"{dept_stats['max']:.0f}"]
        })
        st.dataframe(stats_df, use_container_width=True, hide_index=True)
    
    with col2:
        st.write("**Coverage Metrics**")
        coverage_data = {
            'Metric': [
                'Total Departments', 
                'Avg Hospitals/Dept', 
                'Departments with 1-5 hospitals',
                'Departments with 6+ hospitals',
                'Geographic Span (Lat)',
                'Geographic Span (Lon)'
            ],
            'Value': [
                hospitales['Departamento'].nunique(),
                f"{len(hospitales) / hospitales['Departamento'].nunique():.1f}",
                (hospitales['Departamento'].value_counts() <= 5).sum(),
                (hospitales['Departamento'].value_counts() > 5).sum(),
                f"{hospitales[lat_col].max() - hospitales[lat_col].min():.1f}°",
                f"{hospitales[lon_col].max() - hospitales[lon_col].min():.1f}°"
            ]
        }
        st.dataframe(pd.DataFrame(coverage_data), use_container_width=True, hide_index=True)

def show_static_maps_department_analysis(hospitales, stats_dept):
    """Tab 2: Static Maps & Department Analysis"""
    st.header("🗺️ Static Maps & Department Analysis")
    st.subheader("Static Maps Created with GeoPandas")

    # Mapas estáticos embebidos (usando rutas robustas)
    map_files = [
        ("Hospital Distribution",          f("data/mapa_distribucion_general.png")),
        ("Districts with Zero Hospitals",  f("data/mapa_distritos_sin_hospitales.png")),
        ("Hospital Concentration",         f("data/mapa_concentracion_distritos.png")), 
        ("Top 10 Districts",               f("data/mapa_top10_distritos.png")),
    ]

    for i in range(0, len(map_files), 2):
        cols = st.columns(2)
        for j, (title, file_path) in enumerate(map_files[i:i+2]):
            with cols[j]:
                st.write(f"**{title}**")
                if os.path.exists(file_path):
                    st.image(file_path, use_container_width=True)
                else:
                    st.error(f"Static map not found:\n{file_path}")
                    try:
                        st.caption("Contenido de /data:")
                        st.code("\\n".join(sorted(p.name for p in Path(f('data')).glob('*'))))
                    except Exception:
                        pass
                    st.info("Run Notebook 2 to generate static maps")

    st.subheader("📊 Department Analysis")
    dept_counts = hospitales['Departamento'].value_counts()

    col1, col2 = st.columns(2)

    with col1:
        st.write("**Department Summary Table**")
        dept_summary = pd.DataFrame({
            'Department': dept_counts.index,
            'Hospitals': dept_counts.values,
            'Percentage': (dept_counts.values / dept_counts.sum() * 100).round(1)
        })
        st.dataframe(dept_summary, use_container_width=True, hide_index=True)

    with col2:
        st.write("**Bar Chart - Top Departments**")
        top_10_dept = dept_counts.head(10)
        fig_bar = px.bar(
            x=top_10_dept.values,
            y=top_10_dept.index,
            orientation='h',
            title="Top 10 Departments by Hospital Count",
            labels={'x': 'Number of Hospitals', 'y': 'Department'}
        )
        fig_bar.update_layout(height=400, yaxis={'categoryorder': 'total ascending'})
        st.plotly_chart(fig_bar, use_container_width=True)

def show_dynamic_maps():
    """Tab 3: Dynamic Maps"""
    st.header("🌍 Dynamic Maps")
    st.subheader("🗺️ Interactive Map Selection")

    map_option = st.selectbox(
        "Select a dynamic map to visualize:",
        ["National Choropleth + Markers", "Lima Proximity Analysis", "Loreto Proximity Analysis"]
    )

    # Mapear opciones a archivos (rutas robustas)
    map_files = {
        "National Choropleth + Markers": f("data/mapa_nacional_hospitales.html"),
        "Lima Proximity Analysis":       f("data/mapa_lima_proximidad.html"),
        "Loreto Proximity Analysis":     f("data/mapa_loreto_proximidad.html"),
    }

    map_descriptions = {
        "National Choropleth + Markers": """
**National Folium choropleth + markers:**
- Choropleth map showing hospital density by district
- Interactive marker cluster with all hospital points
- Popup information for each hospital
""",
        "Lima Proximity Analysis": """
**Folium proximity map for Lima:**
- Urban context with high hospital density
- 10 km buffer analysis around selected population center
- Hospitals inside the radius
""",
        "Loreto Proximity Analysis": """
**Folium proximity map for Loreto:**
- Amazonian context with geographic dispersion
- 10 km buffer analysis around selected population center
- Hospitals inside the radius
"""
    }

    st.markdown(map_descriptions[map_option])
    map_file = map_files[map_option]

    if os.path.exists(map_file):
        with open(map_file, 'r', encoding='utf-8') as fhtml:
            html = fhtml.read()
        st.components.v1.html(html, height=600)
    else:
        st.error(f"Dynamic map not found:\n{map_file}")
        try:
            st.caption("Contenido de /data:")
            st.code("\\n".join(sorted(p.name for p in Path(f('data')).glob('*'))))
        except Exception:
            pass
        st.info("Run Notebook 4 to generate interactive maps")

if __name__ == "__main__":
    main()
'''
    # Guardar archivo
    ruta_app = Path("../app/app.py")
    ruta_app.parent.mkdir(parents=True, exist_ok=True)
    ruta_app.write_text(app_code, encoding="utf-8")
    print(f"✅ App.py final creado: {ruta_app}")
    return str(ruta_app)

# Crear aplicación final
app_path = crear_app_streamlit_final()


✅ App.py final creado: ..\app\app.py


In [11]:
def crear_requirements_final():
    """Requirements optimizado para el dashboard"""
    
    requirements_content = """streamlit>=1.28.0
pandas>=1.5.0
geopandas>=0.12.0
plotly>=5.11.0
matplotlib>=3.6.0
seaborn>=0.12.0
shapely>=1.8.0
"""
    
    ruta_req = '../app/requirements.txt'
    with open(ruta_req, 'w') as f:
        f.write(requirements_content)
    
    print(f"✅ Requirements final: {ruta_req}")
    return ruta_req

# Crear requirements
req_path = crear_requirements_final()

✅ Requirements final: ../app/requirements.txt


In [18]:
def validar_archivos_necesarios():
    """Valida que todos los archivos estén presentes"""
    
    print("🔍 VALIDANDO ARCHIVOS NECESARIOS...")
    print("=" * 40)
    
    archivos_requeridos = [
        # App principal
        "../app/app.py",
        "../app/requirements.txt",
        
        # Datos procesados
        "data/hospitales_procesados.geojson",
        
        # Mapas estáticos (PNG)
        "data/mapa_distribucion_general.png",
        "data/mapa_distritos_sin_hospitales.png",
        "data/mapa_concentracion_distritos.png",
        "data/mapa_top10_distritos.png",
        
        # Mapas dinámicos (HTML)
        "data/mapa_nacional_hospitales.html",
        "data/mapa_lima_proximidad.html",
        "data/mapa_loreto_proximidad.html"
    ]
    
    archivos_presentes = 0
    for archivo in archivos_requeridos:
        if os.path.exists(archivo):
            print(f"✅ {archivo}")
            archivos_presentes += 1
        else:
            print(f"❌ {archivo} - FALTANTE")
    
    print(f"\n📊 Resultado: {archivos_presentes}/{len(archivos_requeridos)} archivos presentes")
    
    if archivos_presentes == len(archivos_requeridos):
        print("🎉 DASHBOARD COMPLETO - Listo para ejecutar!")
        print("\n▶️ Ejecutar con:")
        print("cd app")
        print("streamlit run app.py")
    else:
        print("⚠️ Faltan archivos - Ejecuta notebooks faltantes")

# Validar
validar_archivos_necesarios()

🔍 VALIDANDO ARCHIVOS NECESARIOS...
✅ ../app/app.py
✅ ../app/requirements.txt
❌ data/hospitales_procesados.geojson - FALTANTE
❌ data/mapa_distribucion_general.png - FALTANTE
❌ data/mapa_distritos_sin_hospitales.png - FALTANTE
❌ data/mapa_concentracion_distritos.png - FALTANTE
❌ data/mapa_top10_distritos.png - FALTANTE
❌ data/mapa_nacional_hospitales.html - FALTANTE
❌ data/mapa_lima_proximidad.html - FALTANTE
❌ data/mapa_loreto_proximidad.html - FALTANTE

📊 Resultado: 2/10 archivos presentes
⚠️ Faltan archivos - Ejecuta notebooks faltantes


In [13]:
def mostrar_instrucciones_ejecucion():
    """Instrucciones paso a paso para ejecutar el dashboard"""
    
    print("🚀 INSTRUCCIONES DE EJECUCIÓN")
    print("=" * 40)
    
    print("📂 1. Anaconda Prompt:")
    print('   cd "C:\\Users\\bianq\\Documents\\Hospitals-Access-Peru"')
    print("   conda activate hospitals")
    print("   cd app")
    print("   streamlit run app.py")
    print()
    
    print("🌐 2. Abrir navegador:")
    print("   http://localhost:8501")
    print()
    
    print("✅ 3. Dashboard debe mostrar:")
    print("   • Tab 1: Data Description")
    print("   • Tab 2: Static Maps & Department Analysis") 
    print("   • Tab 3: Dynamic Maps")
    print()
    
    print("🛑 4. Para detener:")
    print("   Ctrl + C en el terminal")

mostrar_instrucciones_ejecucion()

🚀 INSTRUCCIONES DE EJECUCIÓN
📂 1. Anaconda Prompt:
   cd "C:\Users\bianq\Documents\Hospitals-Access-Peru"
   conda activate hospitals
   cd app
   streamlit run app.py

🌐 2. Abrir navegador:
   http://localhost:8501

✅ 3. Dashboard debe mostrar:
   • Tab 1: Data Description
   • Tab 2: Static Maps & Department Analysis
   • Tab 3: Dynamic Maps

🛑 4. Para detener:
   Ctrl + C en el terminal


---
## ✅ Notebook 5 Completado - Dashboard Final

### 🎯 Dashboard cumple requerimientos exactos del assignment:

**✅ 3 tabs específicos según template:**
- 🗂️ **Data Description**: Unit of analysis, sources, filtering rules
- 🗺️ **Static Maps & Department Analysis**: GeoPandas maps + department table/charts  
- 🌍 **Dynamic Maps**: Folium choropleth + Lima/Loreto proximity

**✅ Mapas embebidos correctamente:**
- Static maps como imágenes PNG en Tab 2
- Dynamic maps como HTML embebidos en Tab 3

**✅ Análisis completo integrado:**
- 232 hospitales procesados y analizados
- Mapas estáticos e interactivos funcionales
- Análisis departamental con tablas y gráficos
